In [ ]:
from pymongo import MongoClient

from data_collection.RegAPI import Reg_API
from APIScraper import CommentParser

# Setup basic MongoDB collection
client = MongoClient()
db = client.regulationsgov
comments_collection = db.comments

In [ ]:
page_size = 20
api = Reg_API(page_size)
parser = CommentParser(api)

pageNum = 1
while True: 
    comments = api.endpoint("/comments").search("climate").page(pageNum).get()
    
    for i, comment in enumerate(comments["data"]):
        comment_data = parser.get_comment_data(comment)
        print(f"(pg {pageNum}) {i+1}/{page_size}: ", comment_data['_id'], end="")
        print(" "*50, end="\r") # Clear line

        ## Uncomment to store in MongoDB
        #comments_collection.insert_one(comment_data) 

    if pageNum > 1: ### Comment out/delete to run for ALL pages
        break       # - Currently stops after 2nd page
    
    if comments["meta"]["hasNextPage"] == False:
        break
    else:
        pageNum += 1